In [1]:
import clickhouse_connect
import numpy as np
import pandas as pd

client = clickhouse_connect.get_client(
    host='index.bostrom.bronbro.io', port=443,
    username='bronbro', password='spac3b0x')

IBC_COIN_NAMES = \
    {
        'ibc/27394FB092D2ECCD56123C74F36E4C1F926001CEADA9CA97EA622B25F41E5EB2': 'uatom in osmosis',
        'ibc/FE2CD1E6828EC0FAB8AF39BAC45BC25B965BA67CCBC50C13A14BD610B0D1E2C4': 'boot in osmosis',
        'ibc/4F3B0EC2FE2D370D10C3671A1B7B06D2A964C721470C305CBB846ED60E6CAA20': 'hydrogen in osmosis',
        'ibc/BCDB35B7390806F35E716D275E1E017999F8281A81B6F128F087EF34D1DFA761': 'tocyb in osmosis',
        'ibc/46B44899322F3CD854D2D46DEEF881958467CDD4B3B10086DA49296BBED94BED': 'ujuno in osmosis',
        'ibc/EA4C0A9F72E2CEDF10D0E7A9A6A22954DB3444910DB5BE980DF59B05A46DAD1C': 'udsm in osmosis',
        'ibc/EA1D43981D5C9A1C4AAEA9C23BB1D4FA126BA9BC7020A25E0AE4AA841EA25DC5': 'weth in osmosis',
        'ibc/65381C5F3FD21442283D56925E62EA524DED8B6927F0FF94E21E0020954C40B5': 'weth.grv in osmosis',
        'ibc/3FF92D26B407FD61AE95D975712A7C319CDE28DE4D80BDC9978D935932B991D7': 'dot in osmosis',
        'ibc/75345531D87BD90BF108BE7240BD721CB2CB0A1F16D4EBA71B09EC3C43E15C8F': 'btc in osmosis',
        'ibc/1E43D59E565D41FB4E54CA639B838FFD5BCFC20003D330A56CB1396231AA1CBA': 'sol in osmosis',
        'ibc/D79E7D83AB399BFFF93433E54FAA480C191248FC556924A2A8351AE2638B3877': 'utia in osmosis',
        'ibc/E610B83FD5544E00A8A1967A2EB3BEF25F1A8CFE8650FE247A8BD4ECA9DC9222': 'reth in osmosis',
        'ibc/CEE970BB3D26F4B907097B6B660489F13F3B0DA765B83CC7D9A0BC0CE220FA6F': 'uflix in osmosis',
        'ibc/E97634A40119F1898989C2A23224ED83FDD0A57EA46B3A094E287288D1672B44': 'ugraviton in osmosis',
        'ibc/6AE98883D4D5D5FF9E50D7130F1305DA2FFA0C652D1DD9C123657C6B4EB2DF8A': 'aevmos in osmosis',
        'ibc/4ABBEF4C8926DDDB320AE5188CFD63267ABBCEFC0583E4AE05D6E5AA2401DDAB': 'usdt in osmosis',

        'ibc/15E9C5CF5969080539DB395FA7D9C0868265217EFC528433671AAF9B1912D159': 'uatom in bostrom',
        'ibc/BA313C4A19DFBF943586C0387E6B11286F9E416B4DD27574E6909CABE0E342FA': 'deprecated uatom in bostrom',
        'ibc/13B2C536BB057AC79D5616B8EA1B9540EC1F2170718CAFF6F0083C966FFFED0B': 'uosmo in bostrom',
        'ibc/43DB7553C43D81CB01E9A2644B49A241314B482C2E56F86E85A6539C60383151': 'pussy in bostrom',
        'ibc/9B45B8C514B76D792BEC4850AE601E0E73CE7D307A567F34038432FC80D74780': 'liquidpussy in bostrom',
        'ibc/8D9262E35CAE362FA74AE05E430550757CF8D842EC1B241F645D3CB7179AFD10': 'ujuno in bostrom',
        'ibc/C23D820C5B6009E544AFC8AF5A2FEC288108AEDBFAEFDBBDD6BE54CC23069559': 'ugraviton in bostrom',
        'ibc/B6CAD3F7469F3FAD18ED2230A6C7B15E654AB2E1B66E1C70879C04FEF874A863': 'weth.grv in bostrom',
        'ibc/4B322204B4F59D770680FE4D7A565DDC3F37BFF035474B717476C66A4F83DD72': 'aevmos in bostrom',
        'ibc/CA5E8F31288514D728AFD1F0533A7F6902AA1192C88C9540F814893C3EAFE244': 'udsm in bostrom'
     }

def query_to_df(query: str) -> pd.DataFrame:
    query_res = client.query(query=query)
    return pd.DataFrame(query_res.result_rows, columns=query_res.column_names)

## By Claimers

In [2]:
query_1 = r'''
SELECT *
FROM (SELECT signer,
             groupArrayIf(passport_nickname, passport_nickname != '')                        as passport_nicknames,
             uniqExactIf(proof_address, proof_address != '')                                 as proof_addresses,
             uniqExactIf(remove_address, remove_address != '')                               as remove_addresses,
             uniqExactIf(gift_claiming_address, gift_claiming_address != '')                 as claim_addresses,
             uniqExactIf(release_address, release_address != '')                             as release_addresses,
             round(sum(gift_amount_gboot), 1)                                                as claim_gboot,
             round(sumIf(gift_amount_gboot, startsWith(gift_claiming_address, '0x')), 1)     as claim_eth_gboot,
             round(sumIf(gift_amount_gboot, startsWith(gift_claiming_address, 'cosmos')), 1) as claim_cosmos_gboot,
             round(sumIf(gift_amount_gboot, startsWith(gift_claiming_address, 'osmo')), 1)   as claim_osmosis_gboot,
             round(sumIf(gift_amount_gboot, startsWith(gift_claiming_address, 'terra')), 1)  as claim_terra_gboot,
             claim_gboot * 0.34 - release_gboot                                              as releasable_gboot,
             round(sum(released_gboot), 1)                                                   as release_gboot
      FROM (SELECT signer,
                   arrayZip(JSONExtractArrayRaw(messages), JSONExtractArrayRaw(raw_log)) as messages_logs_zip,
                   arrayJoin(messages_logs_zip)                                          as message_log_tuple,
                   message_log_tuple.1                                                   as message_row,
                   message_log_tuple.2                                                   as raw_log_item,
                   trim(BOTH '"' FROM
                        replaceAll(replaceAll(message_row, '\\', ''), '@', ''))          as message,
                   JSONExtractString(message, 'type')                                    as message_type,
                   JSONExtractString(message, 'sender')                                  as message_sender,
                   JSONExtractString(message, 'contract')                                as message_contract,
                   JSONExtractString(message, 'msg')                                     as message_msg,
                   JSON_VALUE(message_msg, '$.create_passport.nickname')                 as passport_nickname,
                   JSON_VALUE(message_msg, '$.proof_address.address')                    as proof_address,
                   JSON_VALUE(message_msg, '$.remove_address.address')                   as remove_address,
                   JSON_VALUE(message_msg, '$.release.gift_address')                     as release_address,
                   round(toInt64OrZero(JSON_VALUE(message_msg, '$.claim.gift_amount')) / 1e9,
                         1)                                                              as gift_amount_raw_gboot,
                   JSON_VALUE(message_msg, '$.claim.gift_claiming_address')              as gift_claiming_address,
                   toInt64OrZero(replace(
                           JSON_VALUE(raw_log_item, '$.events[1].attributes[0].value'),
                           'boot', '')) /
                   1e9                                                                   as released_gboot,
                   toInt64OrZero(JSON_VALUE(raw_log_item, '$.events[5].attributes[4].value')) /
                   1e9                                                                   as gift_amount_gboot,
                   date
            FROM (SELECT height,
                         signer,
                         raw_log,
                         messages,
                         date
                  FROM `spacebox`.transaction FINAL
                           LEFT ANY
                           JOIN (
                      SELECT height,
                             toDate(timestamp) as date
                      FROM `spacebox`.block
                          PREWHERE block.height > 10800000
                      ) as b
                                ON transaction.height = b.height
                      PREWHERE transaction.success = true
                      and transaction.height > 10800000
                     )
            WHERE message_type == '/cosmwasm.wasm.v1.MsgExecuteContract'
              and message_contract in ('bostrom1xut80d09q0tgtch8p0z4k5f88d3uvt8cvtzm5h3tu3tsy4jk9xlsfzhxel',
                                       'bostrom16t6tucgcqdmegye6c9ltlkr237z8yfndmasrhvh7ucrfuqaev6xq7cpvek')
            ORDER BY height DESC)
      GROUP BY signer)
-- WHERE release_gboot > 0
ORDER BY release_gboot DESC
'''

gift_by_singer = query_to_df(query_1)

In [3]:
gift_by_singer

,signer,passport_nicknames,proof_addresses,remove_addresses,claim_addresses,release_addresses,claim_gboot,claim_eth_gboot,claim_cosmos_gboot,claim_osmosis_gboot,claim_terra_gboot,releasable_gboot,release_gboot
0,bostrom1l74ggk5tsn7d2l5qv8fxk3zvdc7fpwrud8gnw6,[etwet46w4],3640,3640,1549,1505,1541.3,1541.3,0.0,0.0,0.0,88.842,435.2
1,bostrom1trt5q36aqdx6l2ca3r5hush2hm9vfu699j9ksn,[fbfbhbvcccccc],2147,2141,2142,2143,1020.5,1020.5,0.0,0.0,0.0,12.970,334.0
2,bostrom1k6kyj0wukla82z2a0ufmrj99xmxhxhj22r04at,[blackhole666],497,497,497,497,450.4,450.4,0.0,0.0,0.0,-0.064,153.2
3,bostrom1ulefdrkmfluv5zvhc4nwuw5vtnaspx8sh9wpk3,[proj3proj],1255,1251,1256,1254,391.5,381.7,0.0,3.2,6.5,1.710,131.4
4,bostrom1g4ywagfj8yc03ecjn7ghet5exxa5l4kqwuplv4,[qwrtgwrhwfq],473,473,119,119,366.2,366.2,0.0,0.0,0.0,1.508,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15980,bostrom1ednc63kwckcz6arhzm28f7undd4gqlvygya0ks,[ngnfn554],2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
15981,bostrom19jcktgz30hz8vd637cgcxmmqvrut3k0ky68fv3,[54545454545454545454545454542],2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
15982,bostrom153cw09k759z52lkmh4xxsgy37gtt6splyf8388,[jasvoyager],0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
15983,bostrom1gt4kqr6jsfd987qsgzj8wh7twg8es6ye50qglr,[laloaaaaaaaaa],3,0,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0


In [4]:
gift_by_singer.groupby('release_addresses').agg(
    {'release_gboot': sum, 'signer': np.count_nonzero, 'release_addresses': sum})

,release_gboot,signer,release_addresses
release_addresses,,,
0,0.0,8160,0
1,5654.3,4497,4497
2,4388.8,1559,3118
3,2598.1,554,1662
4,2361.9,285,1140
5,1578.1,183,915
6,1198.2,166,996
7,712.1,123,861
8,2873.0,416,3328


In [5]:
gift_by_singer.sum()

signer                 bostrom1l74ggk5tsn7d2l5qv8fxk3zvdc7fpwrud8gnw6...
passport_nicknames     [etwet46w4, fbfbhbvcccccc, blackhole666, proj3...
proof_addresses                                                    59605
remove_addresses                                                   21218
claim_addresses                                                    25266
release_addresses                                                  27319
claim_gboot                                                      78597.2
claim_eth_gboot                                                  68580.5
claim_cosmos_gboot                                                5187.9
claim_osmosis_gboot                                               4005.5
claim_terra_gboot                                                  807.4
releasable_gboot                                                2971.848
release_gboot                                                    23751.2
dtype: object

## Transfers after Gift

In [6]:
query_transfers = f'''
WITH send as (SELECT address_from       as sender,
                     address_to         as receiver,
                     denom,
                     sum(amount_coin)   as amount,
                     count(amount_coin) as send_cnt
              FROM (
                       SELECT address_from,
                              address_to,
                              arrayJoin(JSONExtractArrayRaw(coins))       as coin,
                              JSON_VALUE(coin, '$.denom')                 as denom,
                              toInt64OrZero(JSON_VALUE(coin, '$.amount')) as amount_coin
                       FROM `spacebox`.send_message
                           PREWHERE send_message.height > 10800000)
              GROUP BY (address_from,
                        address_to,
                        denom)
              ORDER BY amount DESC),
     ibc_transfers as (SELECT sender,
                              receiver,
                              denom,
                              sum(amount_coin)   as amount,
                              count(amount_coin) as send_cnt
                       FROM (
                                SELECT sender,
                                       receiver,
                                       JSON_VALUE(coin, '$.denom')                 as denom,
                                       toInt64OrZero(JSON_VALUE(coin, '$.amount')) as amount_coin
                                FROM `spacebox`.transfer_message
                                    PREWHERE transfer_message.height > 10800000)
                       GROUP BY (sender, receiver, denom)
                       ORDER BY amount DESC),
     all_transfers as (SELECT *
                       FROM send

                       UNION ALL

                       SELECT *
                       FROM ibc_transfers),
     price as (SELECT 'boot' as denom, 1.0 as price
               UNION ALL
               SELECT 'hydrogen' as denom, 0.29 as price
               UNION ALL
               SELECT 'milliampere' as denom, 1545.9 as price
               UNION ALL
               SELECT 'millivolt' as denom, 14921.0 as price
               UNION ALL
               SELECT 'ibc/15E9C5CF5969080539DB395FA7D9C0868265217EFC528433671AAF9B1912D159' as denom, 799.0 as price
               UNION ALL
               SELECT 'ibc/13B2C536BB057AC79D5616B8EA1B9540EC1F2170718CAFF6F0083C966FFFED0B' as denom, 130.0 as price
               UNION ALL
               SELECT 'ibc/B6CAD3F7469F3FAD18ED2230A6C7B15E654AB2E1B66E1C70879C04FEF874A863' as denom, 0.00000018 as price)

SELECT sender,
       receiver,
       denom,
       amount,
       send_cnt,
       amount * price as value
FROM all_transfers
         ANY
         LEFT JOIN price
                   ON price.denom = all_transfers.denom
ORDER BY value DESC'''

transfers_df = query_to_df(query_transfers)
transfers_df['denom'] = transfers_df['denom'].map(lambda x: IBC_COIN_NAMES[x] if x in IBC_COIN_NAMES.keys() else x)

In [7]:
transfers_df

,sender,receiver,denom,amount,send_cnt,value
0,bostrom1qs9w7ry45axfxjgxa4jmuhjthzfvj78sxh5p6e,bostrom182jzjwdyl5fw43yujnlljddgtrkr04dpd30ywp...,boot,97000000000000,2,9.700000e+13
1,bostrom1lsmmv6q0hayv9e9j3sjdaxean2wg77f0t0hth2,osmo1lsmmv6q0hayv9e9j3sjdaxean2wg77f0q8sgll,uosmo in bostrom,68917273203,1543,8.959246e+12
2,bostrom1xszmhkfjs3s00z2nvtn7evqxw3dtus6yr8e4pw,bostrom19t8jeexz8ny0cjm9w7cn82lv6qe6tkqld29csn,hydrogen,30000000000000,3,8.700000e+12
3,bostrom1lsmmv6q0hayv9e9j3sjdaxean2wg77f0t0hth2,osmo1lsmmv6q0hayv9e9j3sjdaxean2wg77f0q8sgll,boot,5059382739626,2308,5.059383e+12
4,bostrom19t8jeexz8ny0cjm9w7cn82lv6qe6tkqld29csn,bostrom1xszmhkfjs3s00z2nvtn7evqxw3dtus6yr8e4pw,uosmo in bostrom,30000000000,1,3.900000e+12
...,...,...,...,...,...,...
25460,bostrom1v6fnklkj5av596xrv9sucdvcgk7llhracu2yqm,pussy1v6fnklkj5av596xrv9sucdvcgk7llhrajs5frj,pussy in bostrom,77689238481900,1,0.000000e+00
25461,bostrom1zy553za8nenzukmv65240323jhuvxzym4pl6w9,pussy1zy553za8nenzukmv65240323jhuvxzymldphdv,pussy in bostrom,142776946318859,2,0.000000e+00
25462,bostrom1cw4d6ayhdfk2wclvy8fvxff26l8naddqc5m3fu,pussy1cw4d6ayhdfk2wclvy8fvxff26l8naddqjc9u24,liquidpussy in bostrom,154451960011957,4,0.000000e+00
25463,bostrom1pss5jer8r00zv6x562uttctug9u96f4kpqg44w,pussy1pss5jer8r00zv6x562uttctug9u96f4ktvkck8,pussy in bostrom,202218100707996,8,0.000000e+00


In [8]:
transfer_after_gift = \
    gift_by_singer[['signer', 'release_addresses', 'release_gboot']].merge(
        transfers_df[['sender', 'receiver', 'value', 'denom', 'amount']], left_on='signer', right_on='sender', how='left')[[
        'signer', 'receiver', 'release_addresses', 'release_gboot', 'value', 'denom', 'amount'
    ]]
transfer_after_gift['receiver_network'] = transfer_after_gift['receiver'].map(lambda x: x.split('1')[0] if type(x) == str else None)
transfer_after_gift[~transfer_after_gift.receiver.isin(transfer_after_gift['signer'])]
# transfer_after_gift

,signer,receiver,release_addresses,release_gboot,value,denom,amount,receiver_network
0,bostrom1l74ggk5tsn7d2l5qv8fxk3zvdc7fpwrud8gnw6,cosmos1l74ggk5tsn7d2l5qv8fxk3zvdc7fpwruw5uqsa,1505,435.2,8.338380e+11,uatom in bostrom,1.043602e+09,cosmos
1,bostrom1l74ggk5tsn7d2l5qv8fxk3zvdc7fpwrud8gnw6,osmo1l74ggk5tsn7d2l5qv8fxk3zvdc7fpwrux00sx0,1505,435.2,3.414896e+10,boot,3.414896e+10,osmo
2,bostrom1l74ggk5tsn7d2l5qv8fxk3zvdc7fpwrud8gnw6,cosmos1l74ggk5tsn7d2l5qv8fxk3zvdc7fpwruw5uqsa,1505,435.2,6.877418e+09,boot,6.877418e+09,cosmos
3,bostrom1trt5q36aqdx6l2ca3r5hush2hm9vfu699j9ksn,bostrom1082rt24yezp0qkln4h4r2xvx84re373qw8a29f,2143,334.0,3.340403e+11,boot,3.340403e+11,bostrom
5,bostrom1ulefdrkmfluv5zvhc4nwuw5vtnaspx8sh9wpk3,osmo1ulefdrkmfluv5zvhc4nwuw5vtnaspx8sudfz7y,1254,131.4,1.306055e+11,boot,1.306055e+11,osmo
...,...,...,...,...,...,...,...,...
18004,bostrom1ednc63kwckcz6arhzm28f7undd4gqlvygya0ks,NaN,0,0.0,NaN,NaN,NaN,None
18005,bostrom19jcktgz30hz8vd637cgcxmmqvrut3k0ky68fv3,NaN,0,0.0,NaN,NaN,NaN,None
18006,bostrom153cw09k759z52lkmh4xxsgy37gtt6splyf8388,NaN,0,0.0,NaN,NaN,NaN,None
18007,bostrom1gt4kqr6jsfd987qsgzj8wh7twg8es6ye50qglr,NaN,0,0.0,NaN,NaN,NaN,None


In [9]:
transfer_after_gift_agg = transfer_after_gift[~transfer_after_gift.receiver.isin(transfer_after_gift['signer'])].groupby(['receiver_network', 'denom'])[['amount', 'value']].sum().rename(columns={'value': 'boot value'})
transfer_after_gift_agg

amount  \
receiver_network denom                                                              
bostrom          boot                                                4.768857e+12   
                 hydrogen                                            1.038138e+13   
                 milliampere                                         1.000000e+03   
                 millivolt                                           1.858310e+05   
                 uatom in bostrom                                    1.045055e+06   
cosmos           boot                                                5.960316e+10   
                 hydrogen                                            2.008429e+10   
                 ibc/CB41EF44BC15C8857B34EA151C8C480706E2A430A67...  8.575000e+05   
                 millivolt                                           4.297000e+03   
                 uatom in bostrom                                    8.255729e+09   
                 uosmo in bostrom                                    5.737400e+07   
desmos           boot                                                2.663975e+06   
evmos            boot                                                1.000001e+06   
                 hydrogen                                            9.309372e+06   
                 uatom in bostrom                                    6.100000e+05   
gravity          boot                                                9.993797e+06   
                 hydrogen                                            4.200000e+01   
                 ibc/1DE3244B072A8939517C17C7B679F3B735B2D9AA6BE...  1.000000e+05   
                 weth.grv in bostrom                                 5.953000e+18   
juno             boot                                                1.000000e+04   
                 hydrogen                                            4.200000e+01   
                 uatom in bostrom                                    2.000000e+06   
                 ujuno in bostrom                                    3.180603e+09   
osmo             boot                                                1.666252e+13   
                 deprecated uatom in bostrom                         1.826700e+08   
                 hydrogen                                            9.893056e+12   
                 ibc/29FC7F418F1EFB5DAB3DD089AADA1FE406DC78633BF...  1.000000e+03   
                 ibc/5F78C42BCC76287AE6B3185C6C1455DFFF8D805B184...  6.975000e+06   
                 ibc/ABF1AAE56C80610AEBAA8EF4DEF7D850E66F4F35AA5...  1.657612e+10   
                 milliampere                                         1.089773e+08   
                 millivolt                                           8.653062e+06   
                 pool906F3CC4C0F4634031990DB81761BD390890F8A8A80...  1.600000e+01   
                 pussy in bostrom                                    2.200000e+11   
                 tocyb                                               2.260643e+09   
                 uatom in bostrom                                    1.942116e+08   
                 udsm in bostrom                                     4.803900e+07   
                 uosmo in bostrom                                    2.540458e+10   
pussy            boot                                                1.768708e+10   
                 hydrogen                                            1.591611e+10   
                 ibc/47160B934644DED2F8E605B0521C6CC06F75A0465D8...  2.000000e+03   
                 liquidpussy in bostrom                              9.300120e+08   
                 milliampere                                         1.423000e+03   
                 millivolt                                           5.450000e+02   
                 pool70D7610CBA8E94B27BAD7806EBD826F5626C486BBF5...  2.401066e+08   
                 poolE479A51F998E2979F89DA9AB897ED45A9AFE4E7DE32...  3.043116e+07   
                 pussy in bostrom                                    4.935636e+1